## Cargar datos

In [ ]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv("contratos_leyes.csv")

# Ver primeros datos
print(df.head())


## Tokenizacion de datos

In [ ]:
from transformers import AutoTokenizer

# Cargar el tokenizador de Llama
modelo_llama = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(modelo_llama)

# Crear ejemplos de entrenamiento
df["texto_modelo"] = df.apply(lambda row: f"Instrucción: Clasifica el siguiente contrato según si cumple con la ley dominicana.\nContrato: {row['texto']}\nRespuesta: {row['cumple_ley']}. {row['explicacion']}", axis=1)

# Tokenizar los textos
train_encodings = tokenizer(list(df["texto_modelo"]), truncation=True, padding="max_length", max_length=512, return_tensors="pt")

# Verificar un ejemplo tokenizado
print(tokenizer.decode(train_encodings["input_ids"][0]))


## Entrenamiento 

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
import torch

# Cargar modelo de Llama
model = AutoModelForCausalLM.from_pretrained(modelo_llama, torch_dtype=torch.float16, device_map="auto")

# Convertir a dataset de entrenamiento
train_dataset = {"input_ids": train_encodings["input_ids"], "labels": train_encodings["input_ids"]}

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir="./resultados_llama",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch"
)

# Definir trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Iniciar entrenamiento
trainer.train()


Testing

In [ ]:
def predecir_contrato(texto):
    """Clasifica un contrato con el modelo ajustado."""
    prompt = f"Instrucción: Clasifica el siguiente contrato según si cumple con la ley dominicana.\nContrato: {texto}\nRespuesta:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prueba con un contrato nuevo
nuevo_contrato = "El vendedor no entregará los documentos del vehículo hasta después de un año."
print("Evaluación del contrato:", predecir_contrato(nuevo_contrato))
